# Overview
This file is used to parse the user-defined `.json` delta files. They specify which parameters and how the parameters in GPU config files should be varied. There are some conversion functions between JSON and the GPU config files in the Library section for ease of usage.
# Table of contents
- [How to use](#how-to-use)
    - [Rules](#rules)
- [Library section](#library-section)
- [Test section](#test-section)
    - [Test 2](#test-2)
    - [Test 1](#test-1)

# How to use
1. Use the `config_to_dict()` function to convert a specific GPU config file to JSON format (example name `gpgpu.json`)
2. Create JSON file of deltas (example name `deltas.json`) based on `gpgpu.json`, using the rules below
3. Use `parse_deltas()` to parse these deltas
4. Use `apply_parsed_deltas()` to expand these deltas to simulatable config files

Refer to [Test section](#test-section) to see examples of these steps. Example of these files (except for the expanded deltas) can be found in [./util/tuner/RTX2080_S](./util/tuner/RTX2080_S/).

5. Use the different `test_*.sh` files to simulate all the configs: `test_run.sh` -> `test_status.sh` (optional) -> `test_results.sh` (important: need to run the tests on hardware and have the results already in ./hw_run)
6. Use [test_analysis.ipynb](./test_analysis.ipynb) to aggregate the results and plot

## Rules
The `deltas.json` file specifies what parameters you want to vary. If the following characters `':'`, `'*'`, `'='`, `'|'` exist within the value string of a parameter, they signal to the `parse_deltas()` function to create deltas for that parameter. Currently, it is only possible to vary numeric values.

For example, the following config in the GPU config file:
```
-gpgpu_cache:dl1 S:4:128:64,L:T:m:L:L,A:256:32,16:0,32
```

expands to, in the `gpgpu.json` file:

```
    "gpgpu_cache:dl1": [
        [
            "S",
            4,
            128,
            64
        ],
        [
            "L",
            "T",
            "m",
            "L",
            "L"
        ],
        [
            "A",
            256,
            32
        ],
        [
            16,
            0
        ],
        [
            32
        ]
    ],
```

We shall focus on varying parameters in the first part, which corresponds to `<sector?>:<nsets>:<bsize>:<assoc>` of L1:
```
        [
            "S",
            4,
            128,
            64
        ],
```

The `deltas.json` file is a direct copy of `gpgpu.json` file but with certain changes to specify variation. The characters `':'`, `'*'`, `'='`, `'|'` are used as follows:

- `':'`: Create an arithmetic sequence of `<start>:<step>:<end>`. For example, to change `<nsets>` from 1 to 4 with step size 1 we can use the following (note that 3 is not a power of 2):
```
        [
            "S",
            "1:1:4",
            128,
            64
        ],
```

- `'*'`: Create a geometric sequence of `<start>:<multiplier>:<end>`. For example, to change `<nsets>` from 1 to 128 with multiplier 2 we can use the following:
```
        [
            "S",
            "1*2*128",
            128,
            64
        ],
```

- `'='`: Used with `':'` to keep all values with the `'='` sign varying in the same arithmetic sequence. Limited usefulness, I used only for changing multiple clock frequencies the same way. For example, to change `<nsets>` and `<bsize>` simultaneously from 1 to 4 with step size 1:
```
        [
            "S",
            "=1:1:4",
            "=",
            64
        ],
```

- `'|'`: Used with `'*'` to vary multiple parameters geometrically while keeping the total product the same. Useful to keep the same total cache size while varying different parameters. For example, to vary `<nsets>` between 1 and 16, `<bsize>` between 32 and 256, and `<assoc>` automatically so that the total cache size is constant 32768 bytes:
```
        [
            "S",
            "|32768|1*2*16",
            "|32*2*256",
            "|"
        ],
```

Note that in sectored cache (S) the `<bsize>` must always be 128 bytes, so some simulations will not be run.

# Library section

In [2]:
import os
import re
import json

# Convert numeric strings to numbers
def convert_numbers(data):
    if isinstance(data, dict):
        return {k: convert_numbers(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_numbers(item) for item in data]
    elif isinstance(data, str) and data.isdigit():
        return int(data)
    elif isinstance(data, str):
        try:
            return float(data)
        except ValueError:
            return data
    return data

# Convert configs to dicts (used as JSON files)
def config_to_dict(cfg_path: str, cfg_file: str):
    with open(os.path.join(cfg_path, cfg_file)) as f:
        data = f.read().split("\n")

    data = [line for line in data if "#" not in line and len(line) > 0]

    configs = {}
    for line in data:
        if line.startswith('-'):
            config_name, groups = line[1:].split()
            configs[config_name] = [group.split(':') for group in groups.split(',')]

    return convert_numbers(configs)

# Convert dicts (JSON files) to configs
def dict_to_config(configs: dict, cfg_path: str, cfg_file: str):
    data = []
    for config_name in configs.keys():
        line = f"-{config_name} "
        item = configs[config_name]
        for i in range(len(item)):
            for j in range(len(item[i])):
                line += str(item[i][j])
                if j == len(item[i]) - 1:
                    break
                line += ":"
            
            if i == len(item) - 1:
                break
            line += ","
            
        data.append(line)
    
    if not os.path.exists(cfg_path):
        os.makedirs(cfg_path)
        
    with open(os.path.join(cfg_path, cfg_file), 'w') as f:
        for line in data:
            f.write(line + '\n')

In [ ]:
import itertools
import numpy as np
import copy

# Parse arithmetic/geometric variation (:, *)
def parse_range(string):
    if ':' in string:
        start, step, end = map(int, string.split(':'))
        return list(range(start, end + 1, step))
    if '*' in string:
        start, step, end = map(int, string.split('*'))
        values = []
        while start <= end:
            values.append(start)
            start *= step
        return values
    return []

# Parse structure (=, |)
def parse_structure(lst: list[str]):
    values = []
    if len(lst) > 0:
        if lst[0][0] == '=':
            vals = parse_range(lst[0][1:])
            values = [[val] * len(lst) for val in vals]
            
        if lst[0][0] == '|':
            individual_vals = []
            product = lst[0].split('|')[1]
            individual_vals.append(parse_range(lst[0].split('|')[-1]))
            for i in range(len(lst) - 2):
                individual_vals.append(parse_range(lst[i + 1].split('|')[-1]))
                
            if ':' in product or '*' in product:
                products = parse_range(product)
                for p in products:
                    values += [list(tup) + [int(p / np.prod(tup))]
                        for tup in list(itertools.product(*individual_vals))
                        if np.prod(tup) < p]
            else:
                product = int(product)
                values = [list(tup) + [int(product / np.prod(tup))]
                        for tup in list(itertools.product(*individual_vals))
                        if np.prod(tup) < product]
    
    return values

# Convert user-defined delta dict/JSON to parsed delta dict/JSON
def parse_deltas(deltas: dict):
    count = 0
    p_deltas = {}
    for config_name in deltas.keys():
        for i in range(len(deltas[config_name])):
            structure = []
            for j in range(len(deltas[config_name][i])):
                item = deltas[config_name][i][j]
                if isinstance(item, str):
                    results = None
                    if item[0] == '|' or item[0] == '=':
                        structure.append(item)
                        if j == len(deltas[config_name][i]) - 1 or \
                            (deltas[config_name][i][j + 1][0] != '|' and
                            deltas[config_name][i][j + 1][0] != '='):
                            results = parse_structure(structure)
                            count += len(results)
                            structure = []
                            
                    elif ':' in item or '*' in item:
                        if len(structure) > 0:
                            structure = []
                        results = [[val] for val in parse_range(item)]
                        count += len(results)
                    
                    if results is not None:
                        if config_name not in p_deltas.keys():
                            p_deltas[config_name] = []
                        size = len(results[0])
                        p_deltas[config_name].append([
                            i,
                            list(range(j - size + 1, j + 1)),
                            results
                        ])
    
    print(f"Total: {count} configs")
    return p_deltas

# Expand parsed delta dict/JSON to many config files
# They are indexed (ID) sequentially from 0
def apply_parsed_deltas(configs: dict, p_deltas: dict, cfg_dlt_path: str):
    index = 0
    for config_name in p_deltas.keys():
        for entry in p_deltas[config_name]:
            i = entry[0]
            original_vals = copy.deepcopy(configs[config_name][i])
            for vals in entry[2]:
                for j in range(len(entry[1])):
                    configs[config_name][i][entry[1][j]] = vals[j]
                dict_to_config(configs, cfg_dlt_path, f"{str(index).zfill(4)}.config")
                index += 1
                
            configs[config_name][i] = original_vals

# Test section

## Test 1

In [7]:
# Path to config file (gpgpu.config)
cfg_path = "util/tuner/RTX2080_S"
cfg_file = "gpgpusim.config"

# Path to JSON file
jsn_file = "gpgpusim.json"
configs = config_to_dict(cfg_path, cfg_file)
with open(os.path.join(cfg_path, jsn_file), 'w') as f:
    json.dump(configs, f, indent=4)

# ! Copy gpgpusim.json to deltas.json file and specify parameter variations

# Deltas file
dlt_file = "deltas.json"
with open(os.path.join(cfg_path, dlt_file),) as f:
    deltas = json.load(f)
p_deltas = parse_deltas(deltas)

# Save parsed deltas (not neccessary)
p_dlt_file = "parsed_deltas.json"
with open(os.path.join(cfg_path, p_dlt_file), 'w') as f:
    json.dump(p_deltas, f, indent=4)

# Create folder (<cfg_path>/dlt) of generated config files, sequentially indexed
apply_parsed_deltas(configs, p_deltas, os.path.join(cfg_path, "dlt"))

Total: 150 configs


## Test 2

In [9]:
cfg_path = "util/tuner/RTX2080_S"
cfg_file = "gpgpusim.config"
jsn_file = "gpgpusim.json"
configs = config_to_dict(cfg_path, cfg_file)
with open(os.path.join(cfg_path, jsn_file), 'w') as f:
    json.dump(configs, f, indent=4)

dlt_file = "deltas_cuDGT_test2.json"
with open(os.path.join(cfg_path, dlt_file),) as f:
    deltas = json.load(f)
p_deltas = parse_deltas(deltas)
p_dlt_file = "parsed_deltas_cuDGT_test2.json"
with open(os.path.join(cfg_path, p_dlt_file), 'w') as f:
    json.dump(p_deltas, f, indent=4)
apply_parsed_deltas(configs, p_deltas, os.path.join(cfg_path, "dlt_cuDGT_test2"))

Total: 273 configs
